In [22]:
import os
import pandas as pd
from google.cloud import bigquery

In [23]:
# Set credentials

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/dariaserbichenko/code/DariaSerb/key-gcp/trash-optimizer-479913-91e59ecc96c9.json"

PROJECT = "trash-optimizer-479913"
DATASET = "nantes"
client = bigquery.Client(project=PROJECT)

print("Creating optimized trash collection points table")

Creating optimized trash collection points table


In [25]:
# FIRST QUERY ALIMENTARY_GARBAGE (FOOD WASTE)

print("\n1. Querying alimentary_garbage_clean (food waste)")

query1 = f"""
SELECT
  ROW_NUMBER() OVER () as ID,
  CONCAT('Food Waste - ', COALESCE(commune, 'Nantes')) as Name,
  COALESCE(adresse, 'Address not specified') as Address,
  lon as Longitude,
  lat as Latitude,
  0 as Is_Cardboard_enabled,
  1 as Is_Food_enabled,
  0 as Is_Glass_enabled,
  0 as Is_Metal_enabled,
  0 as Is_Paper_enabled,
  0 as Is_Plastic_enabled,
  0 as Is_Textile_enabled,
  0 as Is_Vegetation_enabled,
  0 as Is_Neon_enabled,
  0 as Is_Cartridge_enabled,
  0 as Is_Lamp_Light_enabled
FROM `{PROJECT}.{DATASET}.alimentary_garbage_clean`
WHERE lat IS NOT NULL AND lon IS NOT NULL
"""


1. Querying alimentary_garbage_clean (food waste)


In [26]:
try:
    df1 = client.query(query1).to_dataframe()
    print(f"Retrieved {len(df1):,} food waste locations")
except Exception as e:
    print(f"Error: {e}")
    df1 = pd.DataFrame()

Retrieved 1,644 food waste locations


In [27]:
# SECOND QUERY ECOPOINTS (USING ACTUAL COLUMNS FOUND)

print("\n2. Querying ecopoints with actual columns")

# From inspection: columns are ['bois', 'carton', 'cartouche', 'neon', 'papier', 'textile', 'verre']

query2 = f"""
SELECT
  ROW_NUMBER() OVER () + 10000 as ID,
  CONCAT('Recycling Center - ', COALESCE(nom, commune, 'Ecopoint')) as Name,
  COALESCE(adresse, 'Address not specified') as Address,
  lon as Longitude,
  lat as Latitude,

  -- Use actual columns found
  CASE WHEN UPPER(carton) = 'OUI' THEN 1 ELSE 0 END as Is_Cardboard_enabled,
  0 as Is_Food_enabled,
  CASE WHEN UPPER(verre) = 'OUI' THEN 1 ELSE 0 END as Is_Glass_enabled,
  0 as Is_Metal_enabled,  -- No metal column found
  CASE WHEN UPPER(papier) = 'OUI' THEN 1 ELSE 0 END as Is_Paper_enabled,
  0 as Is_Plastic_enabled,  -- No plastique column
  CASE WHEN UPPER(textile) = 'OUI' THEN 1 ELSE 0 END as Is_Textile_enabled,
  0 as Is_Vegetation_enabled,  -- No vegetal column
  CASE WHEN UPPER(neon) = 'OUI' THEN 1 ELSE 0 END as Is_Neon_enabled,
  CASE WHEN UPPER(cartouche) = 'OUI' THEN 1 ELSE 0 END as Is_Cartridge_enabled,
  0 as Is_Lamp_Light_enabled  -- No ampoule column
FROM `{PROJECT}.{DATASET}.ecopoints`
WHERE lat IS NOT NULL AND lon IS NOT NULL
"""


2. Querying ecopoints with actual columns


In [28]:
try:
    df2 = client.query(query2).to_dataframe()
    print(f"Retrieved {len(df2)} recycling centers with actual waste types")

    # Check acceptance rates
    waste_cols = [col for col in df2.columns if col.startswith('Is_')]
    print(f"- Waste acceptance in recycling centers:")
    for col in waste_cols:
        count = df2[col].sum()
        if count > 0:
            waste_name = col.replace('Is_', '').replace('_enabled', '').replace('_', ' ').title()
            print(f"   {waste_name}: {count}/{len(df2)} locations")

except Exception as e:
    print(f"   ❌ Error: {e}")
    df2 = pd.DataFrame()

Retrieved 15 recycling centers with actual waste types
- Waste acceptance in recycling centers:
   Cardboard: 15/15 locations
   Glass: 14/15 locations
   Paper: 15/15 locations
   Textile: 9/15 locations
   Neon: 8/15 locations
   Cartridge: 15/15 locations


In [29]:
# 3. QUERY COLLECTION_CENTRES_PDL (SIMPLIFIED)

print("\n3. Querying collection_centres_pdl")

# Simple query without problematic columns

query3 = f"""
SELECT
  ROW_NUMBER() OVER () + 20000 as ID, -- Prevents conflicts with IDs from other tables
  CONCAT('Waste Center - ', COALESCE(N_SERVICE, 'PDL Center')) as Name,
  'Contact facility for address' as Address,
  lon as Longitude,
  lat as Latitude,
  1 as Is_Cardboard_enabled,
  1 as Is_Food_enabled,
  1 as Is_Glass_enabled,
  1 as Is_Metal_enabled,
  1 as Is_Paper_enabled,
  1 as Is_Plastic_enabled,
  1 as Is_Textile_enabled,
  1 as Is_Vegetation_enabled,
  1 as Is_Neon_enabled,
  1 as Is_Cartridge_enabled,
  1 as Is_Lamp_Light_enabled
FROM `{PROJECT}.{DATASET}.collection_centres_pdl`
WHERE lat IS NOT NULL AND lon IS NOT NULL
"""


3. Querying collection_centres_pdl


In [30]:
try:
    df3 = client.query(query3).to_dataframe()
    print(f"Retrieved {len(df3):,} waste centers")
except Exception as e:
    print(f"   ❌ Error: {e}")
    df3 = pd.DataFrame()

Retrieved 328 waste centers


In [31]:
# COMBINE AND CREATE FINAL TABLE

print("CREATING FINAL TRASH COLLECTION POINTS TABLE")

all_dataframes = []

if not df1.empty:
    all_dataframes.append(df1)
    print(f"Food waste points: {len(df1):,}")
if not df2.empty:
    all_dataframes.append(df2)
    print(f"Recycling centers: {len(df2)}")
if not df3.empty:
    all_dataframes.append(df3)
    print(f"Waste centers: {len(df3):,}")

if all_dataframes:
    # Combine all data
    combined_df = pd.concat(all_dataframes, ignore_index=True)

    # Reset ID to be sequential
    combined_df['ID'] = range(1, len(combined_df) + 1)

    # Define final structure
    final_columns = [
        'ID', 'Name', 'Address', 'Longitude', 'Latitude',
        'Is_Cardboard_enabled', 'Is_Food_enabled', 'Is_Glass_enabled',
        'Is_Metal_enabled', 'Is_Paper_enabled', 'Is_Plastic_enabled',
        'Is_Textile_enabled', 'Is_Vegetation_enabled', 'Is_Neon_enabled',
        'Is_Cartridge_enabled', 'Is_Lamp_Light_enabled'
    ]

    # Ensure all columns exist
    for col in final_columns:
        if col not in combined_df.columns:
            if col.startswith('Is_'):
                combined_df[col] = 0

    # Convert to proper types
    for col in combined_df.columns:
        if col.startswith('Is_'):
            combined_df[col] = combined_df[col].astype(int)
        elif col in ['Longitude', 'Latitude']:
            combined_df[col] = pd.to_numeric(combined_df[col], errors='coerce')

    # Reorder
    combined_df = combined_df[final_columns]

    total_locations = len(combined_df)
    print(f"FINAL TABLE: {total_locations:,} total trash collection points")

    # Save to CSV
    output_csv = 'trash_collection_points_final_optimized.csv'
    combined_df.to_csv(output_csv, index=False, encoding='utf-8-sig')
    print(f"CSV saved: '{output_csv}'")

    # ===== UPLOAD TO BIGQUERY =====
    print(f"Uploading to BigQuery")
    table_id = f"{PROJECT}.{DATASET}.trash_collection_points"

    job_config = bigquery.LoadJobConfig(
        write_disposition="WRITE_TRUNCATE",
        autodetect=True,
        max_bad_records=100
    )

    try:
        job = client.load_table_from_dataframe(combined_df, table_id, job_config=job_config)
        job.result()

        table = client.get_table(table_id)
        print(f"BigQuery table created: {table_id}")
        print(f"   Rows: {table.num_rows:,}")
        print(f"   Size: {table.num_bytes / (1024*1024):.2f} MB")

    except Exception as e:
        print(f"BigQuery upload failed: {e}")
        print(f"   Data saved locally: '{output_csv}'")


CREATING FINAL TRASH COLLECTION POINTS TABLE
Food waste points: 1,644
Recycling centers: 15
Waste centers: 328
FINAL TABLE: 1,987 total trash collection points
CSV saved: 'trash_collection_points_final_optimized.csv'
Uploading to BigQuery
BigQuery table created: trash-optimizer-479913.nantes.trash_collection_points
   Rows: 1,987
   Size: 0.31 MB


In [32]:
# CREATE DETAILED ANALYSIS

print(f"DETAILED ANALYSIS")


# 1. Facility type breakdown

print(f"\n1. FACILITY TYPES:")
facility_summary = combined_df['Name'].str.extract(r'^(Food Waste|Recycling Center|Waste Center)')[0]
type_counts = facility_summary.value_counts()

for type_name, count in type_counts.items():
    percentage = (count / total_locations) * 100
    print(f"   {type_name:20} {count:6,} locations ({percentage:5.1f}%)")

# 2. Waste type acceptance

    print(f"\n2. WASTE TYPE ACCEPTANCE:")
    waste_cols = [col for col in combined_df.columns if col.startswith('Is_')]

    waste_stats = []
    for col in waste_cols:
        count = combined_df[col].sum()
        percentage = (count / total_locations) * 100
        waste_name = col.replace('Is_', '').replace('_enabled', '').replace('_', ' ').title()
        waste_stats.append((waste_name, count, percentage))

# Sort by most accepted
    waste_stats.sort(key=lambda x: x[1], reverse=True)

    for name, count, pct in waste_stats:
        print(f"   {name:20} {count:6,} locations ({pct:5.1f}%)")

# 3. Geographic coverage

    print(f"\n3. GEOGRAPHIC COVERAGE:")
    if combined_df['Latitude'].notna().any() and combined_df['Longitude'].notna().any():
        min_lat = combined_df['Latitude'].min()
        max_lat = combined_df['Latitude'].max()
        min_lon = combined_df['Longitude'].min()
        max_lon = combined_df['Longitude'].max()

        print(f"   Latitude range:  {min_lat:.4f} to {max_lat:.4f}")
        print(f"   Longitude range: {min_lon:.4f} to {max_lon:.4f}")
        print(f"   Center point:    ({combined_df['Latitude'].mean():.4f}, {combined_df['Longitude'].mean():.4f})")

# 4. Create summary DataFrame

    print(f"\n4. CREATING SUMMARY REPORTS")

# Summary by waste type

    summary_data = []
    for name, count, pct in waste_stats:
        summary_data.append({
            'Waste Type': name,
            'Locations Accepting': count,
            'Percentage': f"{pct:.1f}%",
            'Acceptance Rate': pct/100
        })

    summary_df = pd.DataFrame(summary_data)

    # Summary by facility type
    facility_df = pd.DataFrame({
        'Facility Type': type_counts.index,
        'Count': type_counts.values,
        'Percentage': [f"{(val/total_locations)*100:.1f}%" for val in type_counts.values]
    })

# Save reports

    try:

# Excel report with multiple sheets

        excel_file = 'trash_collection_analysis.xlsx'
        with pd.ExcelWriter(excel_file, engine='openpyxl') as writer:
            combined_df.to_excel(writer, sheet_name='All Locations', index=False)
            summary_df.to_excel(writer, sheet_name='Waste Type Summary', index=False)
            facility_df.to_excel(writer, sheet_name='Facility Type Summary', index=False)

# Add location density analysis

            if combined_df['Latitude'].notna().any():

# Simple density by rounding coordinates

                density_df = combined_df.copy()
                density_df['Lat_rounded'] = density_df['Latitude'].round(2)
                density_df['Lon_rounded'] = density_df['Longitude'].round(2)
                density_summary = density_df.groupby(['Lat_rounded', 'Lon_rounded']).size().reset_index(name='Count')
                density_summary.to_excel(writer, sheet_name='Location Density', index=False)

        print(f"Excel report saved: '{excel_file}'")

    except Exception as e:
        print(f"Excel report failed: {e}")

# Still save CSV summaries

        summary_df.to_csv('waste_type_summary.csv', index=False)
        facility_df.to_csv('facility_type_summary.csv', index=False)
        print(f"CSV summaries saved")

# 5. Quick stats

    print(f"\n5. QUICK STATS:")
    print(f"   Total locations in database: {total_locations:,}")
    print(f"   Locations with coordinates: {combined_df['Latitude'].notna().sum():,}")

# Most common waste types

    top_3 = waste_stats[:3]
    print(f"   Top 3 accepted waste types:")
    for i, (name, count, pct) in enumerate(top_3, 1):
        print(f"     {i}. {name} ({count:,} locations, {pct:.1f}%)")

    print(f"PROCESS COMPLETE")
    print(f"   Total trash collection points: {total_locations:,}")
    print(f"   BigQuery table: {table_id}")
    print(f"   Local files created:")
    print(f"     - {output_csv} (main data)")
    if os.path.exists('trash_collection_analysis.xlsx'):
        print(f"     - trash_collection_analysis.xlsx (analysis report)")

else:
    print("No data was retrieved")

DETAILED ANALYSIS

1. FACILITY TYPES:
   Food Waste            1,644 locations ( 82.7%)

2. WASTE TYPE ACCEPTANCE:
   Food                  1,972 locations ( 99.2%)
   Cardboard               343 locations ( 17.3%)
   Paper                   343 locations ( 17.3%)
   Cartridge               343 locations ( 17.3%)
   Glass                   342 locations ( 17.2%)
   Textile                 337 locations ( 17.0%)
   Neon                    336 locations ( 16.9%)
   Metal                   328 locations ( 16.5%)
   Plastic                 328 locations ( 16.5%)
   Vegetation              328 locations ( 16.5%)
   Lamp Light              328 locations ( 16.5%)

3. GEOGRAPHIC COVERAGE:
   Latitude range:  46.3452 to 48.4623
   Longitude range: -2.5062 to 0.7777
   Center point:    (47.2611, -1.4583)

4. CREATING SUMMARY REPORTS
Excel report saved: 'trash_collection_analysis.xlsx'

5. QUICK STATS:
   Total locations in database: 1,987
   Locations with coordinates: 1,987
   Top 3 accepted wa